## Map Mutations to 3D Structures in the Protein Data Bank

In [74]:
import numpy as np
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset, pdbToUniProt
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd
pd.set_option('display.max_columns', None)  # show all columns
pd.set_option('display.max_columns', None)

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
# Initialize Spark
spark = SparkSession.builder.master("local[4]").appName("2-Map-to-3D").getOrCreate()

#### input parameters

In [3]:
distance_cutoff = 8 # distance cutoff for visualizing interactions

In [4]:
input_file_name = '../analysis/NRF2_pathway/dataframes/step1/mutations_NRF2v2_step1.csv' # contains mutation info in standard format (e.g., chr5:g.149440497C>T)
input_file_name2 = '../analysis/NRF2_pathway/dataframes/step1/mutations_NRF2v2_step1_detailed.csv' # contains depmap details of mutation file

output_file_name = '../analysis/NRF2_pathway/dataframes/step2/mutations_NRF2v2_step2A.csv' # mutations mapped to 3D protein structures
coverage_output = '../dataframes/PDB/DF_PDB_coverage_all_human.csv' #Structural coverage of human proteins

## Read 'mutations_NRF2v2_step1.csv' file created in the previous step

In [5]:
df = pd.read_csv(input_file_name,index_col=0)
var_ids = df['var_id'].tolist()


## Create a list of the variants

In [6]:
#pdb_map = g2sDataset.get_position_dataset(var_ids, ref_genome='hgvs-grch37').toPandas()
pdb_map = g2sDataset.get_full_dataset(var_ids, ref_genome='hgvs-grch37').toPandas()

## Filter PDB Chains

### Filter by sequence identity to PDB sequence

In [7]:
pdb_map['seqIdentity'] = pdb_map.identity/(pdb_map.seqTo - pdb_map.seqFrom + 1) * 100
pdb_map = pdb_map[pdb_map.seqIdentity >= 98]

### Filter by taxonomy (human)

Here we use the SIFTS annotation provided by EBI to filter by taxonomy. To learn more about how to [retrieve SIFTS annotation](
https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/demos/datasets/SiftsDataDemo.ipynb).

In [8]:
taxonomyQuery = "SELECT * FROM sifts.pdb_chain_taxonomy WHERE sifts.pdb_chain_taxonomy.scientific_name = 'Homo sapiens'"
taxonomy = pdbjMineDataset.get_dataset(taxonomyQuery).toPandas()


In [109]:
pdb_filtered = pdb_map.merge(taxonomy, left_on=['structureId','chainId'], right_on=['pdbid','chain'], how='inner')
pdb_filtered = pdb_filtered.drop(['pdbid','chain'], axis=1)  # remove redundant columns
pdb_filtered['pdbPosition'] = pdb_filtered['pdbPosition'].astype('str') # must be string
pdb_filtered.head(2)

,alignmentId,bitscore,chainId,error,evalue,exception,identity,identityPositive,message,midlineAlign,path,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,status,timestamp,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity,tax_id,scientific_name,structureChainId
0,47955439,612.068,A,None,0,None,291.0,291.0,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNL KILVAVK...,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLXXXXXKILVAVK...,1,4at5,4at5_A_1,1,296,hgvs-grch37,"[(C, 805, C, 805)]",543,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLCPEQDKILVAVK...,543,145778,838,None,None,2019-04-26,chr9:g.87636250C>T,4AT5,805,C,98.310811,9606,Homo sapiens,4AT5.A
1,48147361,612.453,A,None,0,None,291.0,291.0,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNL KILVAVK...,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLXXXXXKILVAVK...,1,4at5,4at5_A_1,1,296,hgvs-grch37,"[(C, 805, C, 789)]",543,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLCPEQDKILVAVK...,527,477721,822,None,None,2019-04-26,chr9:g.87636250C>T,4AT5,805,C,98.310811,9606,Homo sapiens,4AT5.A


In [110]:
chains = set(pdb_filtered.structureChainId)


## Get PDB to UniProt Residue Mappings

Download PDB to UniProt mappings and filter out residues that were not observed in the 3D structure.

In [111]:
up = pdbToUniProt.get_cached_residue_mappings().filter("pdbResNum IS NOT NULL").filter("uniprotNum IS NOT NULL")
up_map = up.filter(up.structureChainId.isin(chains)).toPandas()
up_map['uniprotNum'] = up_map.uniprotNum.astype('int') 

In [60]:
pdb_filtered = pdb_filtered.merge(up_map, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','pdbResNum'], how='inner')

In [61]:
pdb_filtered.head(2)

,alignmentId,bitscore,chainId,error,evalue,exception,identity,identityPositive,message,midlineAlign,path,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,status,timestamp,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity,tax_id,scientific_name,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum
0,47955439,612.068,A,None,0,None,291.0,291.0,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNL KILVAVK...,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLXXXXXKILVAVK...,1,4at5,4at5_A_1,1,296,hgvs-grch37,"[(C, 805, C, 805)]",543,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLCPEQDKILVAVK...,543,145778,838,None,None,2019-04-26,chr9:g.87636250C>T,4AT5,805,C,98.310811,9606,Homo sapiens,4AT5.A,805,266,Q16620,789
1,48147361,612.453,A,None,0,None,291.0,291.0,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNL KILVAVK...,None,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLXXXXXKILVAVK...,1,4at5,4at5_A_1,1,296,hgvs-grch37,"[(C, 805, C, 789)]",543,DTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLCPEQDKILVAVK...,527,477721,822,None,None,2019-04-26,chr9:g.87636250C>T,4AT5,805,C,98.310811,9606,Homo sapiens,4AT5.A,805,266,Q16620,789


#### Re-merge DepMap details with PDB filtered data

In [13]:
depmap = pd.read_csv(input_file_name2,index_col=0)

In [23]:
aa = []
for i in depmap.Protein_Change:
    if pd.notnull(i):
        aa.append(i.split('.')[1][0])
    else:
        aa.append('')
depmap['Protein_Change_AA'] = aa

In [62]:
columns = ['variationId','pdbId','structureChainId','pdbPosition','pdbAminoAcid','pdbResNum','uniprotId','seqAlign']

pdb_filtered = pdb_filtered[columns].merge(depmap,left_on='variationId',right_on='var_id')

In [88]:
ind_remove = pdb_filtered.query('pdbAminoAcid != Protein_Change_AA').index

tmp = pdb_filtered[['variationId','structureChainId','pdbAminoAcid','Protein_Change_AA','pdbResNum','Protein_Change']].drop_duplicates()
tmp2 = tmp.query('pdbAminoAcid != Protein_Change_AA')
tmp3 = tmp.query('pdbAminoAcid == Protein_Change_AA')
a = len(tmp2[~tmp2.variationId.isin(tmp3.variationId)].variationId.unique())
b = np.true_divide((len(tmp2)),(len(tmp)))
print("removing %s unique variants, and is %s percent of the total dataframe, as amino acids differ"%(str(a),str(np.round(b,2))))


pdb_filtered.drop(index=ind_remove,inplace=True)

removing 3 unique variants, and is 0.31 percent of the total dataframe, as amino acids differ


In [91]:
pdb_filtered[(pdb_filtered.structureChainId=='3QFB.A')&(pdb_filtered.variationId=='chr12:g.104713303C>T')].pdbResNum.unique()

array(['160'], dtype=object)

## Get Sequence Coverage Data from UniProt

UniProt id, preferred gene name, and sequence length (see column names for RESTful web services: https://www.uniprot.org/help/uniprotkb_column_names).

In [92]:
taxonomy_id = 9606
columns = 'id,genes(PREFERRED),length'

In [93]:
url = f'https://www.uniprot.org/uniprot/?query=organism:{taxonomy_id}&columns={columns}&format=tab'

In [94]:
unp = pd.read_csv(url, sep='\t')
unp.rename(columns={'Gene names  (primary )': 'GENE'}, inplace=True)  ## create name without spaces
unp.head(2)

,Entry,GENE,Length
0,Q6ZS62,COLCA1,124
1,P14384,CPM,443


In [95]:
print('Unique proteins: ', len(unp['Entry'].unique()), 'for organism:', taxonomy_id)
print('Unique genes   : ', len(unp['GENE'].unique()), 'for organism:', taxonomy_id)

Unique proteins:  173199 for organism: 9606
Unique genes   :  26528 for organism: 9606


### Get UniProt segments covered by PDB residues

Get continuous segments of the UniProt sequence covered by PDB residues from EBI SIFTS project (https://www.ebi.ac.uk/pdbe/docs/sifts/)

In [96]:
sifts_url = 'http://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/tsv/uniprot_segments_observed.tsv.gz'

In [97]:
segments = pd.read_csv(sifts_url, sep='\t', skiprows=1)

Calculate length of each continuous segment. A chain may have one or more segments.

In [98]:
segments['SEG_LENGTH'] = segments['SP_END'] - segments['SP_BEG'] + 1

Create a unique key for each chain. Use upper case for PDB IDs. Note, Chain IDs are case sensitive!

In [99]:
segments['PDB_CHAIN_ID'] = segments['PDB'].str.upper()  + "." + segments['CHAIN']
segments = segments[['PDB_CHAIN_ID','SP_PRIMARY','SP_BEG','SP_END','SEG_LENGTH']]

### Calculate coverage from the intersection between the two dataframes

In [100]:
coverage = segments.merge(unp, left_on=['SP_PRIMARY'], right_on=['Entry'])

Calculate coverage per segment

In [101]:
coverage['coverage'] = coverage['SEG_LENGTH'] / coverage['Length']

Calculate coverage per PDB chain

In [102]:
chain_cov = coverage.groupby(['PDB_CHAIN_ID','SP_PRIMARY','GENE']).sum()[['coverage']]
chain_cov = chain_cov[(chain_cov['coverage'] <= 1.0)]  # there are a few cases where coverage > 1 (e.g., P69905, P01579, Q15848)
chain_cov = chain_cov.reset_index()  # convert grouped dataframe to a regular dataframe


In [103]:
print('Unique chains    :', chain_cov.shape[0])
print('Unique proteins  :', len(chain_cov['SP_PRIMARY'].unique()))
print('Unique genes     :', len(chain_cov['GENE'].unique()))
print('Average coverage :', chain_cov['coverage'].mean())
print('Median coverage  :', chain_cov['coverage'].median())

Unique chains    : 100277
Unique proteins  : 6742
Unique genes     : 6633
Average coverage : 0.5511119230799677
Median coverage  : 0.5898617511520737


In [104]:
chain_cov[chain_cov['PDB_CHAIN_ID']=='4Z94.G']

,PDB_CHAIN_ID,SP_PRIMARY,GENE,coverage
61854,4Z94.G,P06396,GSN,0.159847
61855,4Z94.G,P28289,TMOD1,0.164345
61856,4Z94.G,P29536,LMOD1,0.203333


## Save mappings

In [105]:
pdb_filtered.to_csv(output_file_name, index=False)

In [106]:
chain_cov.to_csv(coverage_output, index=False)

In [107]:
# Shutdown Spark
spark.stop()